## Testing after requests is working with Compute


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import os
from pathlib import Path
from bravado.client import SwaggerClient
from bravado.client import SwaggerClient
from bravado.swagger_model import load_file
from bravado.requests_client import RequestsClient
from bravado.swagger_model import load_file


# from bravado.fido_client import FidoClient

import json
import keyring
import requests
import dataclasses
from requests import Session
from requests.auth import HTTPBasicAuth
from requests.auth import _basic_auth_str


In [3]:
import keyring
import requests
from json import load, loads, dump, dumps
from requests import Session

opc_username = "eric.harris@oracle.com"
opc_password = keyring.get_password("OPC", "gc30003")


In [4]:
proxies = {
  'http': 'http://www-proxy-ash7.us.oracle.com:80',
  'https': 'https://www-proxy-ash7.us.oracle.com:80',
}

In [5]:
## https://medium.com/@betz.mark/validate-json-models-with-swagger-and-bravado-5fad6b21a825
# Validate json models with swagger and bravado
from bravado_core.spec import Spec
from bravado_core.validate import validate_object
from yaml import load, Loader, dump, Dumper

In [6]:
idm_domain_name = 'gc30003'
idm_service_instance_id = '587626604'
iaas_rest_endpoint = r'https://compute.uscom-central-1.oraclecloud.com'
iaas_auth_endpoint = f'{iaas_rest_endpoint}/authenticate/'

print(f'iaas_rest_endpoint: {iaas_rest_endpoint}')
print(f'iaas_auth_endpoint: {iaas_auth_endpoint}\n')

iaas_rest_endpoint: https://compute.uscom-central-1.oraclecloud.com
iaas_auth_endpoint: https://compute.uscom-central-1.oraclecloud.com/authenticate/



In [7]:
### Username/pass setup
idm_domain_username = f'/Compute-{idm_domain_name}/{opc_username}'
idm_service_instance_username = f'/Compute-{idm_service_instance_id}/{opc_username}'
# username = traditional_iaas_username
username = idm_service_instance_username
# basic_auth_cred = _basic_auth_str(username, opc_password)

print(f'idm_domain_username: {idm_domain_username}')
print(f'idm_service_instance_username: {idm_service_instance_username}')
print(f'username: {username}')
# print(f'basic_auth_cred: {basic_auth_cred}')

### END Username/pass setup
json_data={"user":username, "password":opc_password}
print(f'\njson_data: {json_data}')


files = None
params = None

### https://docs.oracle.com/en/cloud/iaas/compute-iaas-cloud/stcsa/SendRequests.html
### Supported Headers shown here
# headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
#                ('Accept', 'application/oracle-compute-v3+directory+json'),
#                ('Accept-Encoding', 'gzip;q=1.0, identity; q=0.5'),
#                ('Content-Encoding', 'deflate'),
#                ('Cookie', '<Set from /authenticate>')
#                ])


headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
               ('Accept', 'application/oracle-compute-v3+directory+json'),
               ])

print(f'headers: {headers}')

idm_domain_username: /Compute-gc30003/eric.harris@oracle.com
idm_service_instance_username: /Compute-587626604/eric.harris@oracle.com
username: /Compute-587626604/eric.harris@oracle.com

json_data: {'user': '/Compute-587626604/eric.harris@oracle.com', 'password': 'P@ll4dium!'}
headers: {'Content-Type': 'application/oracle-compute-v3+json', 'Accept': 'application/oracle-compute-v3+directory+json'}


In [8]:
requests_client = RequestsClient()
requests_client.session.headers.update(headers)
requests_client.session.proxies.update(proxies)



print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
requests_client.session.headers.update(headers)
print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")


requests_client.session.headers before update: {'User-Agent': 'python-requests/2.18.4', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/oracle-compute-v3+directory+json', 'Connection': 'keep-alive', 'Content-Type': 'application/oracle-compute-v3+json'}

requests_client.session.headers after update: {'User-Agent': 'python-requests/2.18.4', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/oracle-compute-v3+directory+json', 'Connection': 'keep-alive', 'Content-Type': 'application/oracle-compute-v3+json'}



In [9]:
cwd = os.getcwd()
spec_file_path = Path().joinpath('open_api_definitions/iaas_instances.json').resolve()
print(f'spec_file_path exists: {spec_file_path.exists()}, spec_file_path: {spec_file_path}')

#### http://bravado.readthedocs.io/en/latest/advanced.html#loading-swagger-json-by-file-path
## needed for: client = SwaggerClient.from_url('file:///some/path/swagger.json')
spec_file_uri = f"file:///{spec_file_path}"
print(f'spec_file_uri: {spec_file_path}')


spec_file_path exists: True, spec_file_path: C:\Users\eharris\devel\gc3-query\gc3_query\var\scratchpad\status_update_01\open_api_definitions\iaas_instances.json
spec_file_uri: C:\Users\eharris\devel\gc3-query\gc3_query\var\scratchpad\status_update_01\open_api_definitions\iaas_instances.json


In [10]:

spec_dict = load_file(spec_file_path)
spec_dict['schemes']
print(f"Original spec: spec_dict['schemes']: {spec_dict['schemes']}")
spec_dict['schemes'] = ['https']
print(f"Spec after scheme update: spec_dict['schemes']: {spec_dict['schemes']}")


Original spec: spec_dict['schemes']: ['https']
Spec after scheme update: spec_dict['schemes']: ['https']


In [11]:
swagger_spec = Spec.from_dict(spec_dict=spec_dict, 
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client, 
                                         config={'also_return_response': True}
        )

C:\Apps\python\python_3.6\envs\psmcli\lib\site-packages\bravado_core\spec.py:146: Warning: config also_return_response is not a recognized config key
  category=Warning,


In [12]:
swagger_spec.api_url

'https://compute.uscom-central-1.oraclecloud.com'

In [13]:
# swagger_client = SwaggerClient.from_spec(spec_dict=load_file(spec_file_path), 
#                                          origin_url=iaas_rest_endpoint,
#                                          http_client=requests_client, 
#                                          config={'also_return_response': True})

## Client Configuration
[https://bravado.readthedocs.io/en/latest/configuration.html#client-configuration](https://bravado.readthedocs.io/en/latest/configuration.html#client-configuration)

<pre>
from bravado.client import SwaggerClient, SwaggerFormat

my_super_duper_format = SwaggerFormat(...)

config = {
    # === bravado config ===

    # Determines what is returned by the service call.
    'also_return_response': False,

    # === bravado-core config ====

    #  validate incoming responses
    'validate_responses': True,

    # validate outgoing requests
    'validate_requests': True,

    # validate the swagger spec
    'validate_swagger_spec': True,

    # Use models (Python classes) instead of dicts for #/definitions/{models}
    'use_models': True,

    # List of user-defined formats
    'formats': [my_super_duper_format],

}

client = SwaggerClient.from_url(..., config=config)

</pre>



In [14]:
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict, 
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client, 
                                         config={'also_return_response': True,
                                                'validate_responses': False,})

In [15]:
swagger_spec.definitions

{'DailyWeeklyInterval': abc.DailyWeeklyInterval,
 'HourlyInterval': abc.HourlyInterval,
 'Interval': abc.Interval}

<pre>

DailyWeeklyInterval?

Init signature: DailyWeeklyInterval(**kwargs)
Docstring:     
Attributes:

daysOfWeek: list of string - Days of the week to run a backup
timeOfDay: string - Time of the day to run a backup
userTimeZone: string - Any IANA time zone. e.g: America/Los_Angeles
Init docstring:
Initialize from property values in keyword arguments.

:param \**kwargs: Property values by name.
File:           c:\apps\python\python_3.6\envs\psmcli\lib\abc.py
Type:           ModelMeta

</pre>

<pre>
  "definitions" : {
    "DailyWeeklyInterval" : {
      "allOf" : [ {
        "$ref" : "#/definitions/Interval"
      }, {
        "type" : "object",
        "properties" : {
          "userTimeZone" : {
            "type" : "string",
            "description" : "Any IANA time zone. e.g: America/Los_Angeles"
          },
          "timeOfDay" : {
            "pattern" : "([01]?[0-9]|2[0-3]):[0-5][0-9]",
            "type" : "string",
            "description" : "Time of the day to run a backup"
          },
          "daysOfWeek" : {
            "items" : {
              "enum" : [ "MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY" ],
              "type" : "string"
            },
            "type" : "array",
            "description" : "Days of the week to run a backup"
          }
        },
        "description" : "Backup Interval running once a day on certain days of the week"
      } ]
    },
</pre>

In [16]:
DailyWeeklyInterval = swagger_spec.definitions['DailyWeeklyInterval']
DailyWeeklyInterval?

In [17]:
daily_weekly_interval = DailyWeeklyInterval(userTimeZone='America/Los_Angeles', 
                                            daysOfWeek=['MONDAY', 'FRIDAY'],
                                            timeOfDay='11:00'
                                           )
dir(daily_weekly_interval)
print(f"userTimeZone={daily_weekly_interval.userTimeZone}, daysOfWeek={daily_weekly_interval.daysOfWeek}, timeOfDay={daily_weekly_interval.timeOfDay} ")

userTimeZone=America/Los_Angeles, daysOfWeek=['MONDAY', 'FRIDAY'], timeOfDay=11:00 


In [18]:
daily_weekly_interval = swagger_client.get_model('DailyWeeklyInterval')
daily_weekly_interval

abc.DailyWeeklyInterval

In [19]:
swagger_client.Instances.resource.operations

{'discoverRootInstance': Operation(discoverRootInstance),
 'discoverInstance': Operation(discoverInstance),
 'listInstance': Operation(listInstance),
 'updateInstance': Operation(updateInstance),
 'deleteInstance': Operation(deleteInstance),
 'getInstance': Operation(getInstance)}

## discoverInstance 

** From swagger_client **
<pre>
Type:           Operation
String form:    Operation(discoverInstance)
File:           c:\apps\python\python_3.6\envs\psmcli\lib\site-packages\bravado_core\operation.py
Docstring:      <no docstring>
Init docstring:
Swagger operation defined by a unique (http_method, path_name) pair.

:type swagger_spec: :class:`Spec`
:param path_name: path of the operation. e.g. /pet/{petId}
:param http_method: get/put/post/delete/etc
:param op_spec: operation specification in dict form
</pre>


** From swagger.json instances_swagger_18.1.2.json **

<pre>
    "/instance/{container}" : {
      "get" : {
        "tags" : [ "Instances" ],
        "summary" : "Retrieve Names of all Instances and Subcontainers in a Container",
        "description" : "Retrieves the names of objects and subcontainers that you can access in the specified container.<p><b>Required Role: </b>To complete this task, you must have the <code>Compute_Operations</code> role. If this role isn't assigned to you or you're not sure, then ask your system administrator to ensure that the role is assigned to you in Oracle Cloud My Services. See <a target=\"_blank\" href=\"http://www.oracle.com/pls/topic/lookup?ctx=stcomputecs&id=MMOCS-GUID-54C2E747-7D5B-451C-A39C-77936178EBB6\">Modifying User Roles</a> in <em>Managing and Monitoring Oracle Cloud</em>.",
      <b>  "operationId" : "discoverInstance", </b>
        "responses" : {
          "200" : {
            "headers" : {
              "set-cookie" : {
                "type" : "string",
                "description" : "The cookie value is returned if the session is extended"
              }
            },
            "description" : "OK. See <a class=\"xref\" href=\"Status%20Codes.html\">Status Codes</a> for information about other possible HTTP status codes.",
           <b> "schema" : {
              "$ref" : "#/definitions/Instance-discover-response"
            }</b>
          }
        },
        "consumes" : [ "application/oracle-compute-v3+json" ],
        "produces" : [ "application/oracle-compute-v3+directory+json" ],
        "parameters" : [ {
         <b> "name" : "container", </b>
          "in" : "path",
         <b> "description" : "Specify <code>/Compute-<i>identityDomain</i>/<i>user</i>/</code> to retrieve the names of objects that you can access. Specify <code>/Compute-<i>identityDomain</i>/</code> to retrieve the names of containers that contain objects that you can access.",</b>
          "required" : true,
          "type" : "string"
        }, {
          "name" : "Cookie",
          "in" : "header",
          "type" : "string",
          "description" : "The Cookie: header must be included with every request to the service. It must be set to the value of the set-cookie header in the response received to the POST /authenticate/ call."
        } ]
      }
    },
</pre>




In [20]:
op = swagger_client.Instances.resource.operations['discoverInstance']
op_api_url = op.swagger_spec.api_url
print(f"discoverInstance Operation: {op}, discoverInstance.api_url: {op_api_url}")


discoverInstance Operation: Operation(discoverInstance), discoverInstance.api_url: https://compute.uscom-central-1.oraclecloud.com


In [21]:
instances = swagger_client.Instances


```
class SwaggerClient(object):
    """A client for accessing a Swagger-documented RESTful service.

    :type swagger_spec: :class:`bravado_core.spec.Spec`
    """

    def __init__(self, swagger_spec, also_return_response=False):
        self.__also_return_response = also_return_response
        self.swagger_spec = swagger_spec

    @classmethod
    def from_url(cls, spec_url, http_client=None, request_headers=None, config=None):
        """Build a :class:`SwaggerClient` from a url to the Swagger
        specification for a RESTful API.

        :param spec_url: url pointing at the swagger API specification
        :type spec_url: str
        :param http_client: an HTTP client used to perform requests
        :type  http_client: :class:`bravado.http_client.HttpClient`
        :param request_headers: Headers to pass with http requests
        :type  request_headers: dict
        :param config: Config dict for bravado and bravado_core.
            See CONFIG_DEFAULTS in :module:`bravado_core.spec`.
            See CONFIG_DEFAULTS in :module:`bravado.client`.

        :rtype: :class:`bravado_core.spec.Spec`
        """
        log.debug(u"Loading from %s", spec_url)
        http_client = http_client or RequestsClient()
        loader = Loader(http_client, request_headers=request_headers)
        spec_dict = loader.load_spec(spec_url)

        # RefResolver may have to download additional json files (remote refs)
        # via http. Wrap http_client's request() so that request headers are
        # passed along with the request transparently. Yeah, this is not ideal,
        # but since RefResolver has new found responsibilities, it is
        # functional.
        if request_headers is not None:
            http_client.request = inject_headers_for_remote_refs(
                http_client.request, request_headers)

        return cls.from_spec(spec_dict, spec_url, http_client, config)

    @classmethod
    def from_spec(cls, spec_dict, origin_url=None, http_client=None,
                  config=None):
        """
        Build a :class:`SwaggerClient` from a Swagger spec in dict form.

        :param spec_dict: a dict with a Swagger spec in json-like form
        :param origin_url: the url used to retrieve the spec_dict
        :type  origin_url: str
        :param config: Configuration dict - see spec.CONFIG_DEFAULTS

        :rtype: :class:`bravado_core.spec.Spec`
        """
        http_client = http_client or RequestsClient()

        # Apply bravado config defaults
        config = dict(CONFIG_DEFAULTS, **(config or {}))

        also_return_response = config.pop('also_return_response', False)
        swagger_spec = Spec.from_dict(
            spec_dict, origin_url, http_client, config,
        )
        return cls(swagger_spec, also_return_response=also_return_response)
```

In [22]:
print(f"instances: {instances}, idm_service_instance_username: {idm_service_instance_username}")

instances: <bravado.client.ResourceDecorator object at 0x0000020349036F28>, idm_service_instance_username: /Compute-587626604/eric.harris@oracle.com


In [23]:
container = f"{idm_service_instance_username}/"
print(f"container: {container}")

container: /Compute-587626604/eric.harris@oracle.com/


In [24]:
discover_instance = instances.discoverInstance(container=container)

In [25]:
discover_instance_result = discover_instance.result()

HTTPNotFound: 404 Not Found: Response specification matching http status_code 404 not found for operation Operation(discoverInstance). Either add a response specification for the status_code or use a `default` response.

###### nope, discover_instance_result = discover_instance.result()

We're now getting:

```
ConnectionError: HTTPSConnectionPool(host='compute.uscom-central-1.oraclecloud.com', port=443): Max retries exceeded with url: /instance/%2FCompute-587626604%2Feric.harris%40oracle.com%2F (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000021B45D240B8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

```

%2F is ASCII for '/'
/instance/%2FCompute-587626604%2Feric.harris%40oracle.com%2F -> /instance//Compute-587626604/eric.harris%40oracle.com/

<pre>
        "parameters" : [ {
         <b> "name" : "container", </b>
          "in" : "path",
         <b> "description" : "Specify <code>/Compute-<i>identityDomain</i>/<i>user</i>/</code> to retrieve the names of objects that you can access. Specify <code>/Compute-<i>identityDomain</i>/</code> to retrieve the names of containers that contain objects that you can access.",</b>
          "required" : true,
          "type" : "string"
        }, {
          "name" : "Cookie",
          "in" : "header",
          "type" : "string",
          "description" : "The Cookie: header must be included with every request to the service. It must be set to the value of the set-cookie header in the response received to the POST /authenticate/ call."
        } ]
      }
    },
</pre>


In [ ]:
container = idm_service_instance_username[1:]
print(f"container: {container}")

In [ ]:
discover_instance = instances.discoverInstance(container=container)


In [ ]:
discover_instance_result = discover_instance.result()

In [ ]:
from bravado.swagger_model import Loader

loader = Loader(requests_client, request_headers=None)
spec_dict = loader.load_spec(spec_file_uri)
spec_dict